**To do : find a way to rassemble similar objects**


In [1]:
import json

In [2]:
txt = """{
    "maps": [
        {
            "id": "blabla",
            "iscategorical": "0"
        },
        {
            "id": "blabla",
            "iscategorical": "0"
        }
    ],
    "masks": {
        "id": "valore"
    },
    "om_points": "value",
    "parameters": {
        "id": "valore"
    }
}"""

In [292]:
txt = """{
    "identity": {
        "local": ":Local",
        "version": 1,
        
    },
    "ciphers": [/*Cipher list to decrypt the share_private_key*/]
}"""

In [3]:
def object_of_level(serial) :
    """
    Look at an object
    extract basic types
    return uncommon types
    """
    KEYS = serial.keys()
    UNCLASSED = []
    CLASSED = []
    cnt = 0
    
    for idx, i in enumerate(KEYS) :
        ob = serial[i]
        ty = None
        if type(ob) == list :
            #empty list ?
            ob = ob[0]
            ty = check_type(ob)
            print("this is a list of ", ty)
            CLASSED.append([ty])
        else :
            ty = check_type(ob)
            print("this is a ", ty)
            CLASSED.append(ty)
            
        if ty == dict :
            UNCLASSED.append(ob)
            cnt += 1
            
    return CLASSED, UNCLASSED
        

def check_type(item) :
    if type(item) == dict :
        return dict
    try :
        f = float(item)
        if f == round(f) :
            return int
        else :
            return float
    except :
        if item == "true" or item == "false" :
            return bool
        else :
            return str
            
def python_repr_type (ty, level, objname):
    if ty == bool : 
        return True
    elif ty == int :
        return 0
    elif ty == float :
        return 0.1
    elif ty == str :
        return '"string"'
    elif ty == dict :
        return "{}{}".format(objname, level)
    return None

#### Writable to python

In [7]:
def item_to_python_text(serial, classname="MyObject_") :
    KEYS = serial.keys()
    cnt = 0
    my_text = "class {}(object) :\n".format(classname)
    UNCLASSED = []
    
    for idx, i in enumerate(KEYS) :
        ob = serial[i]
        if type(ob) == list :
            ob = ob[0]
            ty = check_type(ob)
            tx = python_repr_type(ty, cnt, classname)
            my_text += "    {} = [{}]\n".format(i, tx)

        else :
            ty = check_type(ob)
            tx = python_repr_type(ty, cnt, classname)
            my_text += "    {} = {}\n".format(i, tx)
        
        if type(ob) == dict :
            cnt+=1
            UNCLASSED.append(ob)
        
    for idx, i in enumerate(UNCLASSED) :
        my_text += "\n" + item_to_python_text(i, classname="{}_{}".format(classname, idx))
    
    return my_text



### to ocaml :

In [16]:
def ocaml_representation_type(ty,cnt,classname) :
    if ty == bool : 
        return "bool"
    elif ty == int :
        return "int"
    elif ty == float :
        return "float"
    elif ty == str :
        return "string"
    elif ty == dict :
        return "{}{}".format(classname, cnt)
    return None

In [26]:
def item_to_ocaml_record(serial, classname="MyObject_") :
    KEYS = serial.keys()
    cnt = 0
    my_text = "type {} = ".format(classname) + '{ \n'
    UNCLASSED = []
    
    for idx, i in enumerate(KEYS) :
        ob = serial[i]
        
        if type(ob) == list :
            ob = ob[0]
            ty = check_type(ob)
            tx = ocaml_representation_type(ty, cnt, classname)
            my_text += "    {} : list of {};\n".format(i, tx)

        else :
            ty = check_type(ob)
            tx = ocaml_representation_type(ty, cnt, classname)
            my_text += "    {} : {};\n".format(i, tx)
        
        if type(ob) == dict :
            cnt+=1
            UNCLASSED.append(ob)
        
    my_text += '};;\n'
    
    for idx, i in enumerate(UNCLASSED) :
        my_text += "\n" + item_to_ocaml_record(i, classname="{}_{}".format(classname, idx))
    
    return my_text


In [27]:

print(item_to_ocaml_record(my_extractable_object))

type MyObject_ = { 
    parameters : MyObject_0;
    masks : MyObject_1;
    maps : list of MyObject_2;
    om_points : string;
};;

type MyObject__0 = { 
    id : string;
};;

type MyObject__1 = { 
    id : string;
};;

type MyObject__2 = { 
    iscategorical : int;
    id : string;
};;



### save

In [29]:


def write_my_object(txti, name="testfile.py") :
    with open(name, "w") as file:  
        file.write(txti)
        

In [31]:
my_extractable_object= json.loads(txt)

txt_py_exp = item_to_python_text(my_extractable_object)
write_my_object(txt_py_exp, "create_class.py")

txt_oc_exp = item_to_ocaml_record(my_extractable_object)
write_my_object(txt_oc_exp, "create_class.ml")
